# Task 3: AI-Based Diagnostic Assistant (Inference Engine)

This notebook implements a simple rule-based diagnostic assistant using Forward and Backward Chaining inference methods.

## 1. Setup

No external libraries are strictly required for the core logic, but we'll define helper functions for clarity.

In [ ]:
# No specific imports needed for this basic implementation
import copy # To avoid modifying original facts/rules

## 2. Knowledge Base and Rules

Define the knowledge base as a list of rules. Each rule is a tuple: `(premises, conclusion)`.
- `premises`: A set of symptoms (strings) that must be true.
- `conclusion`: The disease (string) inferred if all premises are true.

In [ ]:
knowledge_base = [
    ({'fever', 'cough', 'sore throat'}, 'Flu'),
    ({'fever', 'rash', 'headache'}, 'Measles'),
    ({'fever', 'headache', 'stiff neck'}, 'Meningitis'),
    ({'cough', 'shortness of breath'}, 'Pneumonia'),
    ({'sore throat', 'runny nose'}, 'Common Cold'),
    ({'fatigue', 'fever'}, 'Possible Infection'), # Intermediate conclusion
    ({'Possible Infection', 'rash'}, 'Viral Infection') # Rule using intermediate conclusion
]

## 3. Forward Chaining Implementation

Forward chaining starts with known facts (symptoms) and applies rules to infer new facts (diseases or intermediate conclusions) until no more rules can be applied.

In [ ]:
def forward_chaining(rules, initial_facts):
    """Performs forward chaining inference."""
    facts = copy.deepcopy(initial_facts)
    inferred_facts_log = [] # Log for visualization
    rule_applied_in_iteration = True

    print("--- Forward Chaining --- ")
    print(f"Initial Facts: {facts}")
    iteration = 1

    while rule_applied_in_iteration:
        rule_applied_in_iteration = False
        print(f"\nIteration {iteration}:")
        newly_inferred_facts_this_iteration = set()

        for premises, conclusion in rules:
            # Check if all premises are in the current set of facts
            # and if the conclusion is not already a fact
            if premises.issubset(facts) and conclusion not in facts:
                facts.add(conclusion)
                newly_inferred_facts_this_iteration.add(conclusion)
                log_entry = f"Applied Rule: {premises} => {conclusion}. Inferred: {conclusion}"
                inferred_facts_log.append(log_entry)
                print(log_entry)
                rule_applied_in_iteration = True
        
        if not newly_inferred_facts_this_iteration:
             print("No new facts inferred in this iteration.")
             
        iteration += 1

    print(f"\nFinal Facts: {facts}")
    return facts, inferred_facts_log

## 4. Backward Chaining Implementation

Backward chaining starts with a potential goal (hypothesis, e.g., a specific disease) and works backward, checking if the rules supporting that goal can be satisfied by the known facts.

In [ ]:
def backward_chaining(rules, facts, goal, indent=''):
    """Performs backward chaining inference recursively."""
    print(f"{indent}Goal: Can we prove '{goal}'? Current Facts: {facts}")

    # Base Case 1: Goal is already a known fact
    if goal in facts:
        print(f"{indent}--> Success: '{goal}' is already in known facts.")
        return True, [f"{indent}Fact: {goal}"]

    # Base Case 2: Goal cannot be proven by any rule
    rules_supporting_goal = [(p, c) for p, c in rules if c == goal]
    if not rules_supporting_goal:
        print(f"{indent}--> Failure: No rule concludes '{goal}'.")
        return False, [f"{indent}Cannot prove: {goal}"]

    # Recursive Step: Try to prove the goal using rules
    print(f"{indent}Trying rules that conclude '{goal}':")
    inference_chain = []
    for premises, conclusion in rules_supporting_goal:
        print(f"{indent}  Rule: {premises} => {conclusion}")
        all_premises_proven = True
        rule_chain = [f"{indent}  Trying Rule: {premises} => {conclusion}"]

        for premise in premises:
            print(f"{indent}    Subgoal: Can we prove premise '{premise}'?")
            proven, premise_chain = backward_chaining(rules, facts, premise, indent + '      ')
            rule_chain.extend(premise_chain)
            if not proven:
                print(f"{indent}    --> Failure: Cannot prove premise '{premise}' for this rule.")
                all_premises_proven = False
                rule_chain.append(f"{indent}  Rule Failed: Premise {premise} not proven.")
                break # Stop checking premises for this rule
            else:
                 print(f"{indent}    --> Success: Premise '{premise}' proven.")
                 rule_chain.append(f"{indent}  Premise {premise} proven.")

        if all_premises_proven:
            print(f"{indent}--> Success: All premises for rule {premises} => {conclusion} proven. Therefore, '{goal}' is proven.")
            inference_chain.extend(rule_chain)
            inference_chain.append(f"{indent}Success: {goal} proven via rule {premises} => {conclusion}")
            return True, inference_chain # Goal proven by this rule
        else:
             inference_chain.extend(rule_chain) # Log the attempt even if it failed

    # If no rule could prove the goal
    print(f"{indent}--> Failure: Could not prove '{goal}' with any applicable rule.")
    inference_chain.append(f"{indent}Failure: {goal} could not be proven by any rule.")
    return False, inference_chain

def run_backward_chaining(rules, initial_facts, potential_diagnoses):
    """Runs backward chaining for a list of potential diagnoses."""
    print("\n--- Backward Chaining ---")
    print(f"Initial Facts: {initial_facts}")
    results = {}
    all_chains = {}
    for diagnosis in potential_diagnoses:
        print(f"\nAttempting to prove: {diagnosis}")
        is_proven, chain = backward_chaining(rules, initial_facts, diagnosis)
        results[diagnosis] = is_proven
        all_chains[diagnosis] = chain
        print(f"Result for {diagnosis}: {'Proven' if is_proven else 'Not Proven'}")
    return results, all_chains

## 5. Run Diagnostic Assistant

Define the patient's symptoms (initial facts) and run both inference methods.

In [ ]:
# Example Patient Symptoms
patient_symptoms = {'fever', 'cough', 'sore throat', 'fatigue'}

# Run Forward Chaining
fc_final_facts, fc_log = forward_chaining(knowledge_base, patient_symptoms)

# Define potential diagnoses to check with Backward Chaining
# (Usually the conclusions of the rules)
potential_diagnoses = {conclusion for _, conclusion in knowledge_base}

# Run Backward Chaining
bc_results, bc_chains = run_backward_chaining(knowledge_base, patient_symptoms, potential_diagnoses)

# Print Backward Chaining Summary
print("\n--- Backward Chaining Summary ---")
for diagnosis, proven in bc_results.items():
    print(f"{diagnosis}: {'Proven' if proven else 'Not Proven'}")
    # Optionally print the detailed chain for proven diagnoses:
    # if proven:
    #     print("  Inference Chain:")
    #     for step in bc_chains[diagnosis]:
    #         print(f"    {step}")

## 6. Explanation

### Approach Used

1.  **Knowledge Representation**: A simple rule-based system was used. Rules connect sets of `premises` (symptoms or intermediate conclusions) to a `conclusion` (disease or another intermediate conclusion).
2.  **Inference Engines**:
    *   **Forward Chaining (Data-Driven)**: 
        - Starts with the known `patient_symptoms` (facts).
        - Iteratively applies rules from the `knowledge_base` whose premises are met by the current set of facts.
        - Adds the conclusions of fired rules to the set of facts.
        - Continues until no new facts can be derived.
        - The final set of facts contains all possible inferences, including potential diagnoses.
    *   **Backward Chaining (Goal-Driven)**:
        - Starts with a specific `goal` (a potential diagnosis).
        - Looks for rules that conclude this goal.
        - Recursively tries to prove the premises of those rules, treating them as subgoals.
        - If a subgoal matches a known fact, it's considered proven.
        - If all premises of a rule are proven, the original goal is proven.
        - This process is repeated for each potential diagnosis.
3.  **Output**: 
    *   Forward chaining outputs the final set of all inferred facts.
    *   Backward chaining outputs whether each specific potential diagnosis could be proven based on the initial facts and provides a trace (inference chain) of how it was proven (or why it failed).

### Comparison of Reasoning Styles

**Forward Chaining:**
*   **Pros**: Finds all possible conclusions derivable from the initial facts. Good when the goal isn't known beforehand or when many conclusions might follow from the data.
*   **Cons**: Can be inefficient if only one specific conclusion is needed, as it might derive many irrelevant facts. Can be less intuitive for tracing how a specific conclusion was reached if the chain is long.

**Backward Chaining:**
*   **Pros**: More focused, as it only explores rules relevant to the specific goal(s). Generally more efficient if the number of possible goals is smaller than the number of initial facts. The recursive nature often provides a clearer explanation path for a specific conclusion.
*   **Cons**: Might perform redundant computations if subgoals are revisited multiple times (can be optimized with memoization). Less suitable if the goal is unknown and you want to discover all possibilities.

### Assumptions Made

1.  **Certainty**: The rules and facts are assumed to be 100% certain (no probabilities or fuzzy logic involved).
2.  **Completeness**: The knowledge base is assumed to contain all relevant rules for the diagnoses considered.
3.  **Correctness**: The rules accurately reflect the relationship between symptoms and diseases.
4.  **Monotonic Reasoning**: Adding new facts or rules doesn't invalidate previous conclusions.